# TP 1 : acquisition d’un signal audio

## Étude de la quantification
### 1/ quantum
On definie le quantum comme étant :
q = 2A/2^N
### 2/ Bruit de quantidication
Avec un arrondisement au plus proche quantum on a:
E(q) = 0
et
1/q de [-q/2;q/2]
### 3/ Facteur de Charge
On def le facteur de charge :
F =  A/sqrt(var(x))
### 4/ Oui


## Rééchantillonnage et repliement spectral

# En séance

Import

In [3]:
import wave
import sys
import os
import struct
import numpy as np
from files.quantif import quantif
from scipy.io import wavfile
import pandas as pd
pd.options.plotting.backend = "plotly"
import plotly.express as px

ModuleNotFoundError: No module named 'scipy'

Creation d'une class pour gérer plus facilement les data

In [2]:
class QuantifierComparator:
    bruit_nom = "Bruit"
    norm_nom = "Normalized"
    def __init__(self, file_path):
        self.f = None
        self.nb_frames,self.base_data = wavfile.read(file_path)
        self.data = self.normalised_data = self.base_data / (2**15)
        self.df = pd.DataFrame()
        self.df[self.norm_nom] = self.data
        self.quant_data = None
        self.quant_step = None
        self.quantized_value = None
        self.n=8
        self.a=1
        self.rsb= None
        self.var_signal = None
        self.var_noice = None

    def quantify(self, n=8, a=1):
        self.n=n
        self.a=a
        [self.quant_data, self.quant_step, self.quantized_value] = quantif(self.normalised_data,a,n)

    def pack_in_df(self):
        self.df[self.norm_nom] = self.data
        if self.quant_data is None:
            print("WARNING : data has not been quantified, using base parameters")
            self.quantify()
        self.df[f"Quantified_n{self.n,}_a{ self.a}"] = self.quant_data
        self.df[self.bruit_nom] = self.df[f"Quantified_n{self.n,}_a{ self.a}"]-self.df[self.norm_nom]

    def reset_dataframe(self):
        self.df = pd.DataFrame()

    def plot(self,plot_range = 300):
        self.df.iloc[:plot_range].plot().show()

    def plot_histo(self,name=None):
        if self.df.empty:
            print("empty dataframe, pack pls")
            return
        if name is not None and name in self.df.columns:
            fig = px.histogram(self.df[name],title=name)
            fig.show()
        else:
            for _i in self.df.columns:
                fig = px.histogram(self.df[_i],title=_i)
                fig.show()

    def calc_rsb(self):
        if self.var_signal is None or self.var_noice is None:
            self.calc_metrics()
        self.rsb = self.var_signal/self.var_signal

    def calc_f(self):
        if self.rsb is None:
            self.calc_rsb()
        self.f = self.a/np.sqrt(self.var_signal)

    def calc_metrics(self):
        if self.bruit_nom not in self.df.columns:
            print(self.df)
            self.pack_in_df()
            print("auto_packing")
        self.var_noice = self.df[self.bruit_nom].var()
        self.var_signal = self.df[self.norm_nom].var()


## 1 - Chargement du signal et normalisation

In [1]:
if os.name == "nt":
    path = "files\\sine440.wav"
else:
    path = "files/sine440.wav"
quantifier = QuantifierComparator(path)
# quantifier.plot()
# quantifier.plot_histo()

NameError: name 'os' is not defined

## 2/ Quantifie for N = 4 and A = 1

In [ ]:
quantifier.quantify(n=4,a=1)
quantifier.pack_in_df()
# quantifier.plot()
# quantifier.plot_histo()

## 3 /Noice properties :

In [ ]:
quantifier.plot_histo(quantifier.bruit_nom)
quantifier.calc_metrics()
print(f"Var of noice : {quantifier.var_noice}")

Mean of noice should be 0:

In [316]:
print(f"Mean of noice: {quantifier.df[quantifier.bruit_nom].mean()}")

Mean of noice: 2.763190237032313e-05


## 4/ RSB pour A constant (A=1) et N entre [1;16]

In [317]:
RSB_var_n = {}
for i in range(1,17):
    quantifier.quantify(n=i,a=1)
    quantifier.pack_in_df()
    quantifier.calc_metrics()
    quantifier.calc_rsb()
    RSB_var_n[i] = quantifier.rsb

In [318]:
print(RSB_var_n)

{1: 1.0, 2: 1.0, 3: 1.0, 4: 1.0, 5: 1.0, 6: 1.0, 7: 1.0, 8: 1.0, 9: 1.0, 10: 1.0, 11: 1.0, 12: 1.0, 13: 1.0, 14: 1.0, 15: 1.0, 16: 1.0}


In [296]:
def quantnall(n=15, a=1):
    quant4b = quantif(data_norm,1,4)
    dff = pd.DataFrame(data_norm,columns=["norm"])
    quant4b = quantif(data_norm,a,n)
    dff["Quant4bit"] = quant4b[0]
    dff["bruitQuant"] = dff["Quant4bit"]-dff["norm"]
    var_x = dff["norm"].var()
    var_b = dff["bruitQuant"].var()
    RSB = var_x/var_b
    return RSB, dff


In [297]:
data_ret = {}
for i in range(2,15):
    data_ret[i] = quantnall(i)

In [ ]:
for i in data_ret.values():
    i[1].iloc[:300].plot().show()

In [299]:
data_reta = {}
for i in range(1,50,):
    data_reta[i] = quantnall(n=8,a=i)

In [ ]:
for i in data_reta.values():
    i[1].iloc[:300].plot().show()

In [ ]:
temp_data=  {}
for i,j in data_reta.items():
    temp_data[i] = j[0]

pd.DataFrame(temp_data.values(),index=temp_data.keys()).plot()

In [302]:
temep_data=  {}
for i,j in data_reta.items():
    temp_data[i] = j[0]

Rééchantillonnage et repliement spectral

In [2]:
freq, data = wavfile.read("files\\rythmique.wav")

NameError: name 'wavfile' is not defined